In [ ]:
import numpy as np
from typing import List, Dict, Any


class FuzzyVIKOR:
    """Реализация метода Fuzzy VIKOR с нуля"""
    
    def calculate(self, 
                  matrix: np.ndarray,
                  weights: np.ndarray,
                  benefit_criteria: List[bool],
                  v: float = 0.5,
                  tradeoff_index = 0,
                  tr_custom_values: np.ndarray = None) -> Dict[str, Any]:
        """
        Алгоритм Fuzzy VIKOR.
        """
        m, n, _ = matrix.shape
        
        print("1. Определение идеальных и надирных значений...")
        ideal_point = np.zeros((n, 3))
        nadir_point = np.zeros((n, 3))
        print(ideal_point.shape)
        for j in range(n):
            for k in range(3):
                if benefit_criteria[j]:
                    ideal_point[j, k] = np.max(matrix[:, j, k])
                    nadir_point[j, k] = np.min(matrix[:, j, k])
                else:
                    ideal_point[j, k] = np.min(matrix[:, j, k])
                    nadir_point[j, k] = np.max(matrix[:, j, k])
        
        
        print("2. Поиск нормализованной нечеткой разницы...")
        normalized = np.ndarray((m, n, 3))
        for i in range(m):
            for j in range(n):
                for k in range(3):
                    if benefit_criteria[j]:
                        normalized[i, j, k] = (ideal_point[j, k] - matrix[i, j, 2 - k]) / (ideal_point[j, 2] - nadir_point[j, 0])
                    else:
                        normalized[i, j, k] = (matrix[i, j, k] - ideal_point[j, 2 - k]) / (nadir_point[j, 2] - ideal_point[j, 0])


        print("3. Расчет значений S и R...")
        S_values = np.zeros((m, 3))
        R_values = np.zeros((m, 3))
        
        for i in range(m):
            for k in range(3):
                s_val = 0
                r_val = -np.inf
                for j in range(n):
                    cur_val = weights[j, k] * normalized[i, j, k]
                    s_val += cur_val
                    if (cur_val > r_val):
                        r_val = cur_val
                
                S_values[i, k] = s_val
                R_values[i, k] = r_val
        
        S_star = np.min(S_values, axis=0)   # array [Sl_min, Sm_min, Sr_min]
        S_bar_r = np.max(S_values[:, 2])    # scalar = max over Sr
        R_star = np.min(R_values, axis=0)
        R_bar_r = np.max(R_values[:, 2])

        # 4. Расчёт Q
        print("4. Расчет значений Q...")
        Q_values = np.zeros((m, 3))
        for i in range(m):
            for k in range(3):
                S_part = (S_values[i, k] - S_star[2 - k]) / (S_bar_r - S_star[0])
                R_part = (R_values[i, k] - R_star[2 - k]) / (R_bar_r - R_star[0])
                Q_values[i, k] = v * S_part + (1 - v) * R_part
        

        # 5. Ранжирование
        print("5. Ранжирование альтернатив...")
        core_ranking = np.argsort(Q_values[:, 1])
        
        sorted_Q = Q_values[core_ranking]


        print("6. Подтверждение ранжирования...")
        fuzzy_ranking_confirmations = np.zeros(m)
        for i in range(m):
            confirmed = True
            for i2 in range(i, m):
                confirmed = sorted_Q[i, 0] <= sorted_Q[i2, 0] and sorted_Q[i, 1] <= sorted_Q[i2, 1] and sorted_Q[i, 2] <= sorted_Q[i2, 2]
                if not confirmed:
                    break
            fuzzy_ranking_confirmations[i] = confirmed
        
        print("7. Дефаззификация...")
        crisp_Q = np.zeros(m)
        crisp_R = np.zeros(m)
        crisp_S = np.zeros(m)
        crisp_weights = np.zeros(n)
        crisp_ideal_point = np.zeros(n)
        crisp_nadir_point = np.zeros(n)
        for i in range(m):
            crisp_Q[i] = (Q_values[i, 0] + 2 * Q_values[i, 1] + Q_values[i, 2]) / 4
            crisp_R[i] = (R_values[i, 0] + 2 * R_values[i, 1] + R_values[i, 2]) / 4
            crisp_S[i] = (S_values[i, 0] + 2 * S_values[i, 1] + S_values[i, 2]) / 4
        for j in range(n):
            crisp_weights[j] = (weights[j, 0] + 2 * weights[j, 1] + weights[j, 2]) / 4
            crisp_ideal_point[j] = (ideal_point[j, 0] + 2 * ideal_point[j, 1] + ideal_point[j, 2]) / 4
            crisp_nadir_point[j] = (nadir_point[j, 0] + 2 * nadir_point[j, 1] + nadir_point[j, 2]) / 4
        
        print("8. Четкое ранжирование альтернатив...")
        crisp_ranking_Q = np.argsort(crisp_Q)
        crisp_ranking_R = np.argsort(crisp_R)
        crisp_ranking_S = np.argsort(crisp_S)

        sorted_crisp_Q = crisp_Q[crisp_ranking_Q]
        sorted_crisp_R = crisp_R[crisp_ranking_R]
        sorted_crisp_S = crisp_S[crisp_ranking_S]

        print("9. Подбор решения...")
        # Допустимое превосходство
        advantage_rate = (sorted_crisp_Q[1] - sorted_crisp_Q[0]) / (sorted_crisp_Q[m - 1] - sorted_crisp_Q[0]) 
        DQ = 1/(m - 1)
        acceptable_advantage = advantage_rate >= DQ
        M_acceptable = 0
        if not acceptable_advantage:
            can_search = True
            for i in range(1, m - 1):
                advantage_rate = (sorted_crisp_Q[i + 1] - sorted_crisp_Q[0]) / (sorted_crisp_Q[m - 1] - sorted_crisp_Q[0])
                can_search = advantage_rate <= DQ
                if not can_search:
                    M_acceptable = i + 1
                    break

        # Допустимая стабильность
        acceptable_stability = True
        for i in range(1, m):
            acceptable_stability = sorted_crisp_R[0] <= sorted_crisp_R[i] and sorted_crisp_S[0] <= sorted_crisp_S[i]
            if not acceptable_stability:
                break
        
        #Подбор решения
        compromise_solution = 0
        if not acceptable_advantage:
            compromise_solution = np.zeros(M_acceptable)
            for i in range(M_acceptable):
                compromise_solution[i] = crisp_ranking_Q[i]
        elif not acceptable_stability:
            compromise_solution = np.zeros(2)
            compromise_solution[0] = crisp_ranking_Q[0]
            compromise_solution[1] = crisp_ranking_Q[1]
        else:
            compromise_solution = np.zeros(1)
            compromise_solution[0] = crisp_ranking_Q[0]
        
        print("10. Расчет уступки четкости...")
        Dk = 0
        if benefit_criteria[tradeoff_index]:
            Dk = ideal_point[tradeoff_index, 2] - nadir_point[tradeoff_index, 0]
        else:
            Dk = nadir_point[tradeoff_index, 2] - ideal_point[tradeoff_index, 0]
        tr = np.zeros(n)
        calculated_tr = np.zeros(n)
        for j in range(n):
            if j == tradeoff_index:
                calculated_tr[j] = 1
                continue
            Dj = 0
            if benefit_criteria[j]:
                Dj = ideal_point[j, 2] - nadir_point[j, 0]
            else:
                Dj = nadir_point[j, 2] - ideal_point[j, 0]
            print(j)
            print(Dj)
            print(crisp_weights[tradeoff_index])
            print(Dk)
            print(crisp_weights[j])
            calculated_tr[j] = (Dk * crisp_weights[j]) / (Dj * crisp_weights[tradeoff_index])

        if not tr_custom_values is None:
            tr = tr_custom_values
        else:
            tr = calculated_tr
                
        print("11. Вычисление весов с учетом уступки")
        new_weights = np.zeros((n, 3))
        for j in range(n):
            if j == tradeoff_index:
                new_weights[j, 0] = crisp_weights[j]
                new_weights[j, 1] = crisp_weights[j]
                new_weights[j, 2] = crisp_weights[j]
                continue
            Dj = 0
            if benefit_criteria[j]:
                Dj = ideal_point[j, 2] - nadir_point[j, 0]
            else:
                Dj = nadir_point[j, 2] - ideal_point[j, 0]
            new_weights[j, 0] = abs((Dj * crisp_weights[j] * tr[j]) / Dk)
            new_weights[j, 1] = abs((Dj * crisp_weights[j] * tr[j]) / Dk)
            new_weights[j, 2] = abs((Dj * crisp_weights[j] * tr[j]) / Dk)
            print(j)
            print(Dj)
            print(crisp_weights[j])
            print(tr[j])
            print(Dk)
            print(new_weights[j, 0])

        #--------------------------------------------------------------
        # print("3. Расчет значений S и R...")
        S_values_tr = np.zeros((m, 3))
        R_values_tr = np.zeros((m, 3))
        
        for i in range(m):
            for k in range(3):
                s_val = 0
                r_val = -np.inf
                for j in range(n):
                    cur_val = new_weights[j, k] * normalized[i, j, k]
                    s_val += cur_val
                    if (cur_val > r_val):
                        r_val = cur_val
                
                S_values_tr[i, k] = s_val
                R_values_tr[i, k] = r_val
        
        S_star_tr = np.min(S_values_tr, axis=0)   # array [Sl_min, Sm_min, Sr_min]
        S_bar_r_tr = np.max(S_values_tr[:, 2])    # scalar = max over Sr
        R_star_tr = np.min(R_values_tr, axis=0)
        R_bar_r_tr = np.max(R_values_tr[:, 2])

        # 4. Расчёт Q
        # print("4. Расчет значений Q...")
        Q_values_tr = np.zeros((m, 3))
        for i in range(m):
            for k in range(3):
                S_part = (S_values_tr[i, k] - S_star_tr[2 - k]) / (S_bar_r_tr - S_star_tr[0])
                R_part = (R_values_tr[i, k] - R_star_tr[2 - k]) / (R_bar_r_tr - R_star_tr[0])
                Q_values_tr[i, k] = v * S_part + (1 - v) * R_part
        

        # 5. Ранжирование
        # print("5. Ранжирование альтернатив...")
        core_ranking_tr = np.argsort(Q_values_tr[:, 1])
        
        sorted_Q_tr = Q_values_tr[core_ranking_tr]


        # print("6. Подтверждение ранжирования...")
        fuzzy_ranking_confirmations_tr = np.zeros(m)
        for i in range(m):
            confirmed = True
            for i2 in range(i, m):
                confirmed = sorted_Q_tr[i, 0] <= sorted_Q_tr[i2, 0] and sorted_Q_tr[i, 1] <= sorted_Q_tr[i2, 1] and sorted_Q_tr[i, 2] <= sorted_Q_tr[i2, 2]
                if not confirmed:
                    break
            fuzzy_ranking_confirmations_tr[i] = confirmed
        
        # print("7. Дефуззификация...")
        crisp_Q_tr = np.zeros(m)
        crisp_R_tr = np.zeros(m)
        crisp_S_tr = np.zeros(m)
        crisp_weights_tr = np.zeros(n)
        for i in range(m):
            crisp_Q_tr[i] = (Q_values_tr[i, 0] + 2 * Q_values_tr[i, 1] + Q_values_tr[i, 2]) / 4
            crisp_R_tr[i] = (R_values_tr[i, 0] + 2 * R_values_tr[i, 1] + R_values_tr[i, 2]) / 4
            crisp_S_tr[i] = (S_values_tr[i, 0] + 2 * S_values_tr[i, 1] + S_values_tr[i, 2]) / 4
        for j in range(n):
            crisp_weights_tr[j] = new_weights[j, 0]
        
        # print("8. Четкое ранжирование альтернатив...")
        crisp_ranking_Q_tr = np.argsort(crisp_Q_tr)
        crisp_ranking_R_tr = np.argsort(crisp_R_tr)
        crisp_ranking_S_tr = np.argsort(crisp_S_tr)

        sorted_crisp_Q_tr = crisp_Q_tr[crisp_ranking_Q_tr]
        sorted_crisp_R_tr = crisp_R_tr[crisp_ranking_R_tr]
        sorted_crisp_S_tr = crisp_S_tr[crisp_ranking_S_tr]
                    
        # print("9. Подбор решения...")
        # Допустимое превосходство
        advantage_rate_tr = (sorted_crisp_Q_tr[1] - sorted_crisp_Q_tr[0]) / (sorted_crisp_Q_tr[m - 1] - sorted_crisp_Q_tr[0]) 
        DQ = 1/(m - 1)
        acceptable_advantage_tr = advantage_rate_tr >= DQ
        M_acceptable_tr = 0
        if not acceptable_advantage_tr:
            can_search = True
            for i in range(1, m - 1):
                advantage_rate_tr = (sorted_crisp_Q_tr[i + 1] - sorted_crisp_Q_tr[0]) / (sorted_crisp_Q_tr[m - 1] - sorted_crisp_Q_tr[0])
                can_search = advantage_rate_tr <= DQ
                if not can_search:
                    M_acceptable_tr = i + 1
                    break

        # Допустимая стабильность
        acceptable_stability_tr = True
        for i in range(1, m):
            acceptable_stability_tr = sorted_crisp_R_tr[0] <= sorted_crisp_R_tr[i] and sorted_crisp_S_tr[0] <= sorted_crisp_S_tr[i]
            if not acceptable_stability_tr:
                break
        
        #Подбор решения
        compromise_solution_tr = 0
        if not acceptable_advantage_tr:
            compromise_solution_tr = np.zeros(M_acceptable_tr)
            for i in range(M_acceptable_tr):
                compromise_solution_tr[i] = crisp_ranking_Q_tr[i]
        elif not acceptable_stability_tr:
            compromise_solution_tr = np.zeros(2)
            compromise_solution_tr[0] = crisp_ranking_Q_tr[0]
            compromise_solution_tr[1] = crisp_ranking_Q_tr[1]
        else:
            compromise_solution_tr = np.zeros(1)
            compromise_solution_tr[0] = crisp_ranking_Q_tr[0]
        #-----------------------------------------------------------
        
        # Формирование результатов
        print("Расчёт завершён!\n")
        
        # Создаём нечёткие числа для результатов (для совместимости с print_results)

        
        return {
            "ideal_point": ideal_point,
            "nadir_point": nadir_point,
            "normalized": normalized,
            "S": S_values,
            "R": R_values,
            "Q": Q_values,
            "core_ranking": core_ranking,
            "fuzzy_ranking_confirmations": fuzzy_ranking_confirmations,
            "crisp_Q": crisp_Q,
            "crisp_R": crisp_R,
            "crisp_S": crisp_S,
            "crisp_weights": crisp_weights,
            "crisp_ideal_point": crisp_ideal_point,
            "crisp_nadir_point": crisp_nadir_point,
            "crisp_ranking_Q": crisp_ranking_Q,
            "crisp_ranking_R": crisp_ranking_R,
            "crisp_ranking_S": crisp_ranking_S,
            "acceptable_advantage": acceptable_advantage,
            "M": M_acceptable,
            "acceptable_stability": acceptable_stability,
            "compromise_solution":compromise_solution,
            "tr": calculated_tr,
            "tr_custom_values": tr_custom_values,
            "tradeoff_index": tradeoff_index,
            "new_weights": new_weights,
            "weights_tr": new_weights,


            "S_tr": S_values_tr,
            "R_tr": R_values_tr,
            "Q_tr": Q_values_tr,
            "core_ranking_tr": core_ranking_tr,
            "fuzzy_ranking_confirmations_tr": fuzzy_ranking_confirmations_tr,
            "crisp_Q_tr": crisp_Q_tr,
            "crisp_R_tr": crisp_R_tr,
            "crisp_S_tr": crisp_S_tr,
            "crisp_weights_tr": crisp_weights_tr,
            "crisp_ranking_Q_tr": crisp_ranking_Q_tr,
            "crisp_ranking_R_tr": crisp_ranking_R_tr,
            "crisp_ranking_S_tr": crisp_ranking_S_tr,
            "acceptable_advantage_tr": acceptable_advantage_tr,
            "M_tr": M_acceptable_tr,
            "acceptable_stability_tr": acceptable_stability_tr,
            "compromise_solution_tr":compromise_solution_tr,
        }
    
    def print_results(self, results: Dict[str, Any], print_fuzzy_details: bool = False):
        """Вывод результатов"""
        print("=" * 50)
        print("РЕЗУЛЬТАТЫ FUZZY VIKOR")
        print("=" * 50)

        print(f"Идеальное значение: {results["ideal_point"]}")
        print(f"Надирное значение: {results["nadir_point"]}")
        
        print("\nДефаззифицированные значения:")
        print("Альт \t|\t Ql \t Qm \t Qr \tCrisp Q\t |\t Sl \t Sm \t Sr \tCrisp S\t |\t Rl \t Rm \t Rr \tCrisp R")
        
        for i in range(len(results["crisp_Q"])):
            ql = results["Q"][i, 0]
            qm = results["Q"][i, 1]
            qr = results["Q"][i, 2]
            qcrisp = results["crisp_Q"][i]

            sl = results["S"][i, 0]
            sm = results["S"][i, 1]
            sr = results["S"][i, 2]
            scrisp = results["crisp_S"][i]

            rl = results["R"][i, 0]
            rm = results["R"][i, 1]
            rr = results["R"][i, 2]
            rcrisp = results["crisp_R"][i]
            print(f"A{i+1:2d} \t |\t{ql:0.3f}\t{qm:0.3f}\t{qr:0.3f}\t{qcrisp:0.3f}\t|\t{sl:0.3f}\t{sm:0.3f}\t{sr:0.3f}\t{scrisp:0.3f}\t|\t{rl:0.3f}\t{rm:0.3f}\t{rr:0.3f}\t{rcrisp:0.3f}")
        
        print("=" * 50)
        print("Ранжирование:")
        print("Ранг \t|\tCore\t|\tConf.\t|\tCrisp Q\tCrisp S\tCrisp R")
        for i in range(len(results["core_ranking"])):
            rank = results["core_ranking"][i] + 1
            confirmation = results["fuzzy_ranking_confirmations"][i]
            crisp_rank_Q = results["crisp_ranking_Q"][i] + 1
            crisp_rank_R = results["crisp_ranking_R"][i] + 1
            crisp_rank_S = results["crisp_ranking_S"][i] + 1
            print(f"{i+1:2d} \t |\tА{rank:2d}\t|\t{confirmation:0.0f}\t|\tА{crisp_rank_Q:2d}\tА{crisp_rank_S:2d}\tА{crisp_rank_R:2d}")
        
        print("=" * 50)
        print("Подобранное решение:")
        print(f"Допустимое превосходство: {results["acceptable_advantage"]}")
        if not results["acceptable_advantage"]:
            print(f"M: {results["M"]}")
        print(f"Допустимая стабильность: {results["acceptable_stability"]}")
        print("Компромиссное решение: ")
        for i in range(len(results["compromise_solution"])):
            solution = int(results["compromise_solution"][i] + 1)
            print(f"A{solution:2d}")
        
        print("=" * 50)

        print("Уступки (trade-off): ")
        print("tr\t|\tЗаданный tr\t|\tНовые веса")
        has_given = not results["tr_custom_values"] is None
        for i in range(len(results["tr"])):
            tr = results["tr"][i]
            new_weight = results["new_weights"][i][0]
            if has_given:
                tr_given = results["tr_custom_values"][i]
                print(f"{tr:0.3f}\t|\t{tr_given:0.3f}\t\t|\t{new_weight:0.3f}")
            else:
                print(f"{tr:0.3f}\t|\t-\t\t|\t{new_weight:0.3f}")

        print("=" * 50)
        print("\nДефаззифицированные значения после уступки:")
        print("Альт \t|\t Ql \t Qm \t Qr \tCrisp Q\t |\t Sl \t Sm \t Sr \tCrisp S\t |\t Rl \t Rm \t Rr \tCrisp R")
        
        for i in range(len(results["crisp_Q_tr"])):
            ql = results["Q_tr"][i, 0]
            qm = results["Q_tr"][i, 1]
            qr = results["Q_tr"][i, 2]
            qcrisp = results["crisp_Q_tr"][i]

            sl = results["S_tr"][i, 0]
            sm = results["S_tr"][i, 1]
            sr = results["S_tr"][i, 2]
            scrisp = results["crisp_S_tr"][i]

            rl = results["R_tr"][i, 0]
            rm = results["R_tr"][i, 1]
            rr = results["R_tr"][i, 2]
            rcrisp = results["crisp_R_tr"][i]
            print(f"A{i+1:2d} \t |\t{ql:0.3f}\t{qm:0.3f}\t{qr:0.3f}\t{qcrisp:0.3f}\t|\t{sl:0.3f}\t{sm:0.3f}\t{sr:0.3f}\t{scrisp:0.3f}\t|\t{rl:0.3f}\t{rm:0.3f}\t{rr:0.3f}\t{rcrisp:0.3f}")
        
        print("=" * 50)
        print("Ранжирование после уступки:")
        print("Ранг \t|\tCore\t|\tConf.\t|\tCrisp Q\tCrisp S\tCrisp R")
        for i in range(len(results["core_ranking_tr"])):
            rank = results["core_ranking_tr"][i] + 1
            confirmation = results["fuzzy_ranking_confirmations_tr"][i]
            crisp_rank_Q = results["crisp_ranking_Q_tr"][i] + 1
            crisp_rank_R = results["crisp_ranking_R_tr"][i] + 1
            crisp_rank_S = results["crisp_ranking_S_tr"][i] + 1
            print(f"{i+1:2d} \t |\tА{rank:2d}\t|\t{confirmation:0.0f}\t|\tА{crisp_rank_Q:2d}\tА{crisp_rank_S:2d}\tА{crisp_rank_R:2d}")
        
        print("=" * 50)
        print("Подобранное решение:")
        print(f"Допустимое превосходство: {results["acceptable_advantage_tr"]}")
        if not results["acceptable_advantage_tr"]:
            print(f"M: {results["M_tr"]}")
        print(f"Допустимая стабильность: {results["acceptable_stability_tr"]}")
        print("Компромиссное решение: ")
        for i in range(len(results["compromise_solution_tr"])):
            solution = int(results["compromise_solution_tr"][i] + 1)
            print(f"A{solution:2d}")


    
    # Тестовые данные
    # matrix = np.array([
    #     [[0.7, 0.8, 0.9], [0.6, 0.7, 0.8], [0.8, 0.9, 1.0]],  # A1
    #     [[0.5, 0.6, 0.7], [0.8, 0.9, 1.0], [0.6, 0.7, 0.8]],  # A2
    #     [[0.8, 0.9, 1.0], [0.5, 0.6, 0.7], [0.7, 0.8, 0.9]],  # A3
    # ])
#     matrix = np.array([
#         [[38.00, 40.01, 48.00], [3.26, 4.08, 4.08], [43, 47, 48], [10, 10, 10]],  # A1
#         [[20, 21.06, 24], [2.57,2.87,2.87], [6, 6, 6], [10, 10, 10]],  # A2
#         [[24.58, 25.87, 29.85], [2.82,2.97,2.97], [38, 42, 50], [1, 1, 1]],  # A3
#         [[44.54, 46.89, 56.27], [2.46,2.73,2.73], [60, 62, 68], [0, 0, 0]],  # A4
#         [[33.33, 33.33, 43.33], [2.25,2.50,2.62], [6, 6, 6], [2, 2, 2]],  # A5
#         [[33.86, 33.86, 42.32], [2.47,2.74,2.85], [6, 6, 6], [3, 3, 3]],  # A6
#     ])
    

# matrix = np.array([
#     [[15.0, 17.0, 40.0],  [60.0, 70.0, 85.0],  [25.0, 30.0, 36.0]],   # A1
#     [[18.0, 21.0, 35.0],  [75.0, 90.0,110.0],  [30.0, 35.0, 42.0]],   # A2
#     [[16.0, 18.0, 32.0],  [85.0,100.0,120.0],  [35.0, 40.0, 50.0]],   # A3
#     [[14.0, 16.0, 28.5],  [50.0, 60.0, 72.0],  [28.0, 33.0, 39.0]],   # A4
#     [[20.0, 23.0, 38.0],  [95.0,115.0,140.0],  [40.0, 48.0, 58.0]]    # A5
# ])
# #     matrix = np.array([
#     [[17.0, 17.0, 17.0],  [70.0, 70.0, 70.0],  [30.0, 30.0, 30.0]],   # A1
#     [[21.0, 21.0, 21.0],  [90.0, 90.0,90.0],   [35.0, 35.0, 35.0]],   # A2
#     [[18.5, 18.5, 18.5],  [100.0,100.0,100.0], [40.0, 40.0, 40.0]],   # A3
#     [[16.0, 16.0, 16.0],  [60.0, 60.0, 60.0],  [33.0, 33.0, 33.0]],   # A4
#     [[23.0, 23.0, 23.0],  [115.0,115.0,115.0], [48.0, 48.0, 48.0]]    # A5
# ])
    
weights = np.array([[1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1]])
benefit_criteria = [True, True, True, False, True]

# Запуск алгоритма
fvikor = FuzzyVIKOR()
results = fvikor.calculate(matrix, weights, benefit_criteria, v=0.625, tr_custom_values=np.array([1, 15, 0.2, 2]))

# Вывод результатов
fvikor.print_results(results, print_fuzzy_details=False)
    

1. Определение идеальных и надирных значений...
(3, 3)
2. Поиск нормализованной нечеткой разницы...
3. Расчет значений S и R...
4. Расчет значений Q...
5. Ранжирование альтернатив...
6. Подтверждение ранжирования...
7. Дефаззификация...
8. Четкое ранжирование альтернатив...
9. Подбор решения...
10. Расчет уступки четкости...
1
90.0
1.0
26.0
1.0
2
33.0
1.0
26.0
1.0
11. Вычисление весов с учетом уступки
1
90.0
1.0
15.0
26.0
51.92307692307692
2
33.0
1.0
0.2
26.0
0.2538461538461539
Расчёт завершён!

РЕЗУЛЬТАТЫ FUZZY VIKOR
Идеальное значение: [[ 20.  23.  40.]
 [ 95. 115. 140.]
 [ 40.  48.  58.]]
Надирное значение: [[14.  16.  28.5]
 [50.  60.  72. ]
 [25.  30.  36. ]]

Дефаззифицированные значения:
Альт 	|	 Ql 	 Qm 	 Qr 	Crisp Q	 |	 Sl 	 Sm 	 Sr 	Crisp S	 |	 Rl 	 Rm 	 Rr 	Crisp R
A 1 	 |	-0.478	0.308	0.992	0.282	|	-0.537	1.276	2.850	1.216	|	0.121	0.545	1.000	0.553
A 2 	 |	-0.560	0.199	0.896	0.184	|	-0.804	0.749	2.417	0.777	|	-0.061	0.394	0.848	0.394
A 3 	 |	-0.646	0.141	0.890	0.132	|	-1.04

In [6]:
import json
def parse_vikor_json(file_path):
    """Parse VIKOR JSON file and extract all variables."""
    
    # Read JSON from file
    with open(file_path, 'r') as f:
        data = json.load(f)
    alternatives = data["alternatives"] 
    criteria = data["criteria"] 
    dms = data["dms"] 
    parameters = data["parameters"] 

    weights_list = [criterion["weight"] for criterion in criteria]
    weights = np.array([weights_list]*3).T

    benefit_criteria = [criterion["type"] == "positive" for criterion in criteria]

    matrix = np.array(dms[0]["scores"])

    v = parameters["v"] 
    tr_custom_values = parameters.get("tr_custom_values")

    if tr_custom_values is not None:
        tr_custom_values = np.array(tr_custom_values)
    return {
        "matrix": matrix,
        "weights": weights,
        "benefit_criteria": benefit_criteria,
        "alternatives": alternatives,
        "criteria": [c["name"] for c in criteria],
        "v": v,
        "tr_custom_values": tr_custom_values,
    }




In [11]:
in_data = parse_vikor_json("mlava.json")
matrix = in_data["matrix"]
weights = in_data["weights"]
benefit_criteria = in_data["benefit_criteria"]
v = in_data["v"]
tr_custom_values = in_data["tr_custom_values"]

fvikor = FuzzyVIKOR()
results = fvikor.calculate(matrix, weights, benefit_criteria, v=v, tr_custom_values=tr_custom_values)
fvikor.print_results(results=results)

1. Определение идеальных и надирных значений...
(4, 3)
2. Поиск нормализованной нечеткой разницы...
3. Расчет значений S и R...
4. Расчет значений Q...
5. Ранжирование альтернатив...
6. Подтверждение ранжирования...
7. Дефаззификация...
8. Четкое ранжирование альтернатив...
9. Подбор решения...
10. Расчет уступки четкости...
1
1.83
1.0
36.27
1.0
2
62.0
1.0
36.27
1.0
3
10.0
1.0
36.27
1.0
11. Вычисление весов с учетом уступки
1
1.83
1.0
15.0
36.27
0.7568238213399504
2
62.0
1.0
0.2
36.27
0.3418803418803419
3
10.0
1.0
2.0
36.27
0.5514199062586159
Расчёт завершён!

РЕЗУЛЬТАТЫ FUZZY VIKOR
Идеальное значение: [[20.   21.06 24.  ]
 [ 3.26  4.08  4.08]
 [ 6.    6.    6.  ]
 [ 0.    0.    0.  ]]
Надирное значение: [[44.54 46.89 56.27]
 [ 2.25  2.5   2.62]
 [60.   62.   68.  ]
 [10.   10.   10.  ]]

Дефаззифицированные значения:
Альт 	|	 Ql 	 Qm 	 Qr 	Crisp Q	 |	 Sl 	 Sm 	 Sr 	Crisp S	 |	 Rl 	 Rm 	 Rr 	Crisp R
A 1 	 |	0.086	0.448	1.000	0.495	|	1.535	2.184	2.897	2.200	|	1.000	1.000	1.000	1.000
A 2